# 0 - étape 1: Installation NLTK et librairies de visualisations

In [ ]:
import sys
# 1... installer NLKT si vous ne l'avez pas déjà fait 
#!{sys.executable} -m pip install nltk

In [ ]:
#1... import the library NLKT
# A new window should open, showing the NLTK Downloader. Click on the File menu and select Change Download Directory. 
# For central installation, set this to C:\nltk_data (Windows), /usr/local/share/nltk_data (Mac), or /usr/share/nltk_data (Unix). 
# Next, download all.
import nltk
#nltk.download()

In [ ]:
#2....installer visualisation libraries
!{sys.executable} -m pip install networkx[default]
!{sys.executable} -m pip install pyvis
!{sys.executable} -m pip install textblob
!{sys.executable} -m pip install wordcloud

# 1 - Corpus - example 1 : Wordnet 

In [ ]:
# Sample usage for wordnet
# WordNet is just a NLTK corpus reader, and can be imported like this:
from nltk.corpus import wordnet as wn

In [ ]:
word="lemme"
synset = wn.synsets(word, lang='fra')
print('The test word is : ', word)
print('Word and Type : ' + synset[0].name())
print('Synonym is: ' + synset[0].lemmas()[0].name())
print('The meaning of the word : ' + synset[0].definition())
print('Example : ' + str(synset[0].examples()))

In [ ]:
from textblob import Word
noeud = Word("word")
print (noeud.synsets[:10])
print (noeud.definitions[:10])


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
G=nx.Graph()

w=noeud.synsets[0]

G.add_node(w.name())
for h in w.hypernyms():
    #print (h)
    G.add_node(h.name())
    G.add_edge(w.name(),h.name())


for h in w.hyponyms():
    #print (h)
    G.add_node(h.name())
    G.add_edge(w.name(),h.name())

print (G.nodes(data=True))
plt.show()
plt.rcParams['figure.figsize'] = [21, 5]
nx.draw(G, width=1, with_labels=True, node_color="#007ed9")
plt.savefig("path.png")

In [ ]:
# to see all other layouts: https://networkx.org/documentation/stable/reference/generated/networkx.drawing.layout.random_layout.html
nx.draw(G, pos=nx.spiral_layout(G))
plt.show()

# 2 - Corpus - example 2 : Guttenberg

## a - Télécharger un livre du site

In [ ]:
# Lets explore the Guttenberg in NLKT !
nltk.download('gutenberg')
nltk.corpus.gutenberg.fileids()

In [ ]:
moby_dick = nltk.corpus.gutenberg.words( 'melville-moby_dick.txt')
len(moby_dick)

In [ ]:
for text in nltk.corpus.gutenberg.fileids():
    print('# of words in ',text,'is: ', len(nltk.corpus.gutenberg.words( text)))

## b - Télécharger un livre du site

In [ ]:
# Lets load a specific book from Guttenberg website 
# you will need to leverage the requests package
import requests
#choose a book in Gutenberg project website the https://www.gutenberg.org/ebooks/5258 and get the reference number of the book, here 5258 !
r = requests.get(r'https://www.gutenberg.org/cache/epub/5258/pg5258.txt')
Zarathoustra_Nietzsche = r.text

# first, remove unwanted new line and tab characters from the text
for char in ["\n", "\r", "\d", "\t"]:
    Zarathoustra_Nietzsche = Zarathoustra_Nietzsche.replace(char, " ")
#print number of characters in the book
print(len(Zarathoustra_Nietzsche))

In [ ]:
#To see the project gutenburg introduction and footnotes
print(Zarathoustra_Nietzsche[0:910]) 
print('-------------------------------------------------') 
print(Zarathoustra_Nietzsche[637986:639986]) 

In [ ]:
# you can also subset for the book text
# (removing the project gutenburg introduction/footnotes)
Zarathoustra_Nietzsche = Zarathoustra_Nietzsche[911:637986]
#print(Zarathoustra_Nietzsche)

## b- Explorer le corpus

In [ ]:
# Lets Tokenize the Text
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
import re

# Choose your Text
text = Zarathoustra_Nietzsche

In [ ]:
tokens = word_tokenize(text, language="french")

In [ ]:
# print the 20 most commons tokens
from collections import Counter
print(Counter(tokens).most_common(40))

In [ ]:
# Lets remove punctiation !
remove = re.sub(r'[^\w\s]', '', text)
#print("updated text with no punctuations :", remove)
tokens = word_tokenize(remove, language="french")
print(Counter(tokens).most_common(40))

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
wordcloud = WordCloud(width= 1000, height = 600, max_words=100,
                      random_state=1, background_color='White', colormap='cubehelix',
                      collocations=False, stopwords = STOPWORDS).generate(text)
plt.figure(figsize=(5, 5))
plt.imshow(wordcloud) 
plt.axis("off")
plt.show()

In [ ]:
# Lets remove Stopwords !
french_stopwords = set(stopwords.words('french'))
filtre_stopfr = lambda text: [token for token in text if token.lower() not in french_stopwords]

tokens_Filtered=filtre_stopfr( tokens)
print(Counter(tokens_Filtered).most_common(20))

In [ ]:
wordcloud = WordCloud(width= 1000, height = 600, max_words=100,
                      random_state=1, background_color='Black', colormap='Paired',
                      collocations=False, stopwords = french_stopwords).generate(text)
plt.figure(figsize=(7, 5))
plt.imshow(wordcloud) 
plt.axis("off")
plt.show()

In [ ]:
import numpy as np
import os
from PIL import Image

folder = "/Users/loubnaserrar/0-Cours Text mining"

data = Image.open(os.path.join(folder, "Nietzsche.jpg"),'r')
#mask = np.array(Image.open("Nietzsche.jpg"))
mask = np.array(data)

mask.shape
# Generating colors from image
image_colors = ImageColorGenerator(mask)

In [ ]:
wordcloud = WordCloud(width= 1000, height = 800, max_words=700,
                      random_state=1, background_color='Lightblue', colormap='winter_r',
                      collocations=False, stopwords = french_stopwords, mask = mask).generate(text)
plt.figure(figsize=(5, 13))
plt.imshow(wordcloud) 
#plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation ='bilinear') # Using the color function to use the image colors
plt.axis("off")
plt.show()

#Pour afficher l'image mask...
plt.imshow(mask) 
plt.axis("off")
plt.show()

In [ ]:
# Do you wantg to exclude anything else?
Stop_words=['plus']
for x in Stop_words:
    french_stopwords.add(x)

In [ ]:
tokens_Filtered=filtre_stopfr( tokens)
print(Counter(tokens_Filtered).most_common(20))

# 3 - Corpus example 4 : Webscrapping

## Web scraping : construire votre corpus directement de Reddit
PRAW, an acronym for "Python Reddit API Wrapper", is a Python package that allows for simple access to Reddit's API. PRAW aims to be easy to use and internally follows all of Reddit's API rules. With PRAW there's no need to introduce sleep calls in your code. Give your client an appropriate user agent and you're set.

#https://scrapingrobot.com/blog/web-scraping-reddit/

### 1 - Installer PRAW : Python Reddit API Wrapper

In [ ]:
!pip install praw

import praw

### 2 - créer une application sur le site Reddit
--> you need to create or log into your account on Reddit.

--> click on “Are you a developer? Create an app…”

https://www.reddit.com/prefs/apps

Suivre le Tuto suivant :
https://towardsdatascience.com/scraping-reddit-data-1c0af3040768


In [ ]:
raddit 


In [ ]:
# noter les identifiants de votre apploication personnelle
my_client_id = ""

my_client_secret = ""

my_user_agent = ""

In [ ]:
reddit = praw.Reddit(client_id=my_client_id, client_secret=my_client_secret, user_agent=my_user_agent)

In [ ]:
# get 10 hot posts from the MachineLearning subreddit
hot_posts = reddit.subreddit('ChatGPT').hot(limit=10)
for post in hot_posts:
    print(post.title)

In [ ]:
# get hottest posts from all subreddits
hot_posts = reddit.subreddit('all').hot(limit=10)
for post in hot_posts:
    print(post.title)

In [ ]:
import pandas as pd
posts = []
ml_subreddit = reddit.subreddit('ChatGPT')
for post in ml_subreddit.hot(limit=10):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])


In [ ]:
posts.head()

## Option B : télécharger votre coprus que vous avez déjà préparer !

In [ ]:
#Download Dataset
import pandas as pd
# Attention changer le chemin d'accées à votre fichier
df = pd.read_csv('D:/Lila/Download/tweets_Greve_2023.csv',encoding='utf-8')

In [ ]:
df.shape

In [ ]:
df.head(5)

In [ ]:
df.tail(5)

In [ ]:
df.nunique() # nunique permet de compter le nombre des valeurs uniques dans DataFrame

In [ ]:
print(df['User'].unique()) # unique renvoie des valeurs uniques comme un Numpy array
print("number of different users in this Twitter corpus is: ", len(df['User'].unique())) 
print("number of different languages used in this Twitter corpus is: ", len(df['Langue'].unique()))

In [ ]:
# vérifions combien sont en français ?
df.groupby(['Langue']).nunique()

In [ ]:
df[df['Langue']=='fr'].count()

In [ ]:
# Créer une copie de l'objet en incluant les données et l'index
# (deep copy) : utile car les changements effectués dans l'original ne sont pas reflétés dans la copie
df_nd = df[['TweetID', 'Date', 'User', 'Tweet', 'Langue']].copy() 
df_nd.shape
print(df_nd)

In [ ]:
df_nd = df_nd[df_nd['Langue']=='fr']
df_nd.shape

In [ ]:
# drop_duplicates : enlever les lignes dupliquées
df_nd = df_nd.drop_duplicates().copy()
df_nd.shape

In [ ]:
df_nd.head()

In [ ]:
df_nd = df_nd.drop(columns=['Langue'])
df_nd.shape

In [ ]:
df_nd.tail()

In [ ]:
# Explore the data : lenght of the Tweets
df_nd['char_len'] = df_nd['Tweet'].apply(lambda x:len(x))
df_nd['word_len'] = df_nd['Tweet'].apply(lambda x:len(x.split(" ")))
df_nd.head()

#### Visualison la densité du corpus !
https://seaborn.pydata.org/generated/seaborn.kdeplot.html
- longueur des Tweets 
- densité : Un graphique d'estimation de la densité du noyau (KDE) est une méthode de visualisation de la distribution des observations dans un ensemble de données, analogue à un histogramme. L'estimation de la densité du noyau représente les données à l'aide d'une courbe de densité de probabilité continue dans une ou plusieurs dimensions.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.displot(data = df_nd, x='char_len', kde=True)

plt.show()

In [ ]:
# A kernel density estimate (KDE) plot is a method for visualizing the distribution of observations in a dataset, analogous to a histogram. KDE represents the data using a continuous probability density curve in one or more dimensions.
# https://seaborn.pydata.org/generated/seaborn.kdeplot.html

sns.displot(data = df_nd, x='word_len', kde=True)

plt.show()